## Trying to plot concentrations surfaces using plotly 

In [1]:
# (*) To communicate with Plotly's server, sign in with credentials file
import plotly.plotly as py

# (*) Useful Python/Plotly tools
import plotly.tools as tls

# (*) Graph objects to piece together plots
from plotly.graph_objs import *

import numpy as np  

from netCDF4 import Dataset

import os 

import sys

In [2]:
#lib_path = os.path.abspath('/ocean/kramosmu/Building_canyon/BuildCanyon/PythonModulesMITgcm') # Add absolute path to my python scripts
lib_path = os.path.abspath('/Users/anakarinarm/SubmarineCanyonsResearch/PythonModulesMITgcm') # Add absolute path to my python scripts

sys.path.append(lib_path)

import ReadOutTools_MITgcm as rout 


In [3]:
#Base case, iso =1 , No 3d diff.
#CanyonGrid='/ocean/kramosmu/MITgcm/TracerExperiments/CNTDIFF/run03/gridGlob.nc'
CanyonGrid='/Volumes/MOBY/NOGMREDI/run03/gridGlob.nc'
CanyonGridOut = Dataset(CanyonGrid)

CanyonState='/Volumes/MOBY/NOGMREDI/run03/stateGlob.nc'
CanyonStateOut = Dataset(CanyonState)

Ptracer='/Volumes/MOBY/NOGMREDI/run03/ptracersGlob.nc'
PtracerOut = Dataset(Ptracer)


In [4]:
# General input
nx = 360
ny = 360
nz = 90
nt = 19 # t dimension size 

z = CanyonStateOut.variables['Z']
#print(z[10])
Time = CanyonStateOut.variables['T']
#print(Time[:])

xc = rout.getField(CanyonGrid, 'XC') # x coords tracer cells

yc = rout.getField(CanyonGrid, 'YC') # y coords tracer cells

bathy = rout.getField(CanyonGrid, 'Depth')

hFacC = rout.getField(CanyonGrid, 'HFacC')
MaskC = rout.getMask(CanyonGrid, 'HFacC')

dxF = rout.getField(CanyonGrid, 'dxF')
drF = CanyonGridOut.variables['drF']

In [5]:
Tr1 = rout.getField(Ptracer,'Tr1') # 


In [27]:
ConcInd = np.argmax(Tr1[5,:,:,:]>=7.4,axis=0)
ConcDepths = np.empty((360,360))
for i in range(360):
    for j in range(360):
        ConcDepths[i,j] = z[ConcInd[i,j]]

ConcDepthsMasked = np.ma.masked_values(ConcDepths,-2.5)

#print(ConcDepths)


[[-157.5 -157.5 -157.5 ..., -157.5 -157.5 -157.5]
 [-157.5 -157.5 -157.5 ..., -157.5 -157.5 -157.5]
 [-157.5 -157.5 -157.5 ..., -157.5 -157.5 -157.5]
 ..., 
 [   nan    nan    nan ...,    nan    nan    nan]
 [   nan    nan    nan ...,    nan    nan    nan]
 [   nan    nan    nan ...,    nan    nan    nan]]


In [29]:
ConcSurf = Surface(
    z=ConcDepthsMasked,  # link the fxy 2d numpy array
    x=xc[1,:],  # link 1d numpy array of x coords
    y=yc[:,1],
    opacity=0.9# link 1d numpy array of y coords
)

# Package the trace dictionary into a data object
data2 = Data([ConcSurf])

In [30]:
# Dictionary of style options for all axes
axis2 = dict(
    showbackground=True, # (!) show axis background
    backgroundcolor="rgb(204, 204, 204)", # set background color to grey
    gridcolor="rgb(255, 255, 255)",       # set grid line color
    zerolinecolor="rgb(255, 255, 255)",   # set zero grid line color
)

# Make a layout object
layout2 = Layout(
    title='Canyon bathymetry', # set plot title
    scene=Scene(  # (!) axes are part of a 'scene' in 3d plots
        xaxis=XAxis(axis2), # set x-axis style
        yaxis=YAxis(axis2), # set y-axis style
        zaxis=ZAxis(axis2)  # set z-axis style
    )
)


In [35]:
## Make a figure object
##fig = Figure(data=data2, layout=layout2)

## (@) Send to Plotly and show in notebook
##py.iplot(fig, filename='s8_surface')

#py.plot([
#    dict(z=-bathy,x=xc[1,:],y=yc[:,1],colorscale="Greys", type='surface'),
#    dict(z=ConcDepths,x=xc[1,:],y=yc[:,1],opacity=0.7, type='surface'),
#    ], auto_open='false', filename='ConcPrueba')

py.iplot([
    dict(z=-bathy[200:-50,100:-100],x=xc[1,100:-100],y=yc[200:-50,1],colorscale="Greys", type='surface'),
    dict(z=ConcDepthsMasked[200:-50,100:-100],x=xc[1,100:-100],y=yc[200:-50,1],opacity=1.0, type='surface'),
    ],auto_open='false', filename='ConcPrueba')
